In [7]:
from models import *
from andi_datasets.models_phenom import models_phenom
from helpersGeneration import *
from helpersPlot import *
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# Define input shape
batch_size = 2
patch_size = 7
embed_dim = 64
num_heads = 4
hidden_dim = 128
num_layers = 6

model_linear = LinearTransformer(patch_size, embed_dim, num_heads, hidden_dim, num_layers)


In [9]:
# number of time steps per trajectory (frames)
T = 200
# number of trajectories
N = 1
# Length of box (pixels), set to 0 to make them start at 0,0
L = 0 
# diffusion coefficient (pixels^2 / s)
D = 5
nposframe = 10

trajs, labels = models_phenom().single_state(N, 
                                L = L,
                                T = T,
                                Ds = [D, 0.0], # Mean and variance
                                alphas = 1)
print(trajs.shape,labels.shape)
# Need to reshape generated trajectories because they are in format (T,N,dim), but we want them in (N,T,dim)
trajs = trajs.transpose(1,0,2)
labels = labels.transpose(1,0,2)
print(trajs.shape,labels.shape)
# Labels are now in format (N,T,3): for each particle T times the tuple (alpha, D, state)
print(f"Particle 0 [alpha, Gen_D, state]: {labels[0,0,:]}")


# convert trajectories of D (pixels/s) to D (micro_m/ms)
trajs = trajs / 100


videos = trajectories_to_video(trajs,nposframe,
                                   center=True,
                                   image_props={"upsampling_factor":10,
                                            "background_intensity": [100,10],
                                            "particle_intensity": [500,20],
                                            "resolution": 100e-9,
                                            "trajectory_unit" : 1000,
                                            "output_size": 7,
                                            "poisson_noise" : -1
                                              })

print(videos.shape)
play_video(videos[0])

videos_t = torch.from_numpy(videos)


(200, 1, 2) (200, 1, 3)
(1, 200, 2) (1, 200, 3)
Particle 0 [alpha, Gen_D, state]: [1. 5. 2.]
(1, 20, 7, 7)
vmin: 73.3399658203125 vmax: 522.4415893554688 mean: 136.63


In [10]:
model_linear(videos_t)

tensor([[[ 1.0186, -0.6870,  0.7142,  ...,  0.6252,  0.5995,  0.0044],
         [ 0.8772, -0.6471,  0.8278,  ...,  0.6288,  0.5094, -0.0152],
         [ 0.7048, -0.8232,  0.7284,  ...,  0.5461,  0.5895,  0.0031],
         ...,
         [ 0.8729, -0.7690,  0.6114,  ...,  0.6853,  0.6503,  0.0107],
         [ 1.0249, -0.8205,  0.6746,  ...,  0.6385,  0.5583, -0.0171],
         [ 0.8441, -0.6492,  0.7212,  ...,  0.7714,  0.5601, -0.0767]]],
       grad_fn=<NativeLayerNormBackward0>)